# NGAME prediction 

In [1]:
#| default_exp 43-1-encoder-parallel-ngame-for-wikiseealso

In [2]:
%load_ext autoreload
%autoreload 2

from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [3]:
#| export
import os,torch, torch.multiprocessing as mp, pickle
from scipy import sparse
from xcai.basics import *
from xcai.models.PPP0XX import DBT009

In [4]:
#| export
os.environ['WANDB_MODE'] = 'disabled'
os.environ['CUDA_VISIBLE_DEVICES'] = '12,13'

## Prediction

In [6]:
#| export
data_dir = '/home/aiscuser/scratch/datasets'
pkl_file = f'{data_dir}/processed/wikiseealso_data_distilbert-base-uncased_xcnlg_ngame.pkl'

with open(pkl_file, 'rb') as file: block = pickle.load(file)

In [7]:
#| export
args = XCLearningArguments(
    output_dir='/home/aiscuser/outputs/43-encoder-parallel-ngame-for-wikiseealso-1-0',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    predict_with_representation=True,
    generation_num_beams=10,
    generation_length_penalty=1.5,
    predict_with_generation=True,
    representation_search_type='BRUTEFORCE',
    output_concatenation_weight=1.0,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    fp16=True,
)

In [8]:
#| export
output_dir = f"/home/aiscuser/scratch/Projects/xc_nlg/outputs/{os.path.basename(args.output_dir)}"
mname = f'{output_dir}/{os.path.basename(get_best_model(output_dir))}'

In [11]:
#| export
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT009.from_pretrained(mname, ig_tok=0, bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1, n_negatives=5, 
                               apply_softmax=True, tie_word_embeddings=False)

In [13]:
#| export
test_dset = block.test.dset.sample(n=2000, seed=50)
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

learn = XCLearner(model=model, args=args, train_dataset=block.train.dset, eval_dataset=test_dset,
                  data_collator=block.collator, compute_metrics=metric)

if __name__ == '__main__':
    mp.freeze_support()
    test_pred = learn.predict(block.test.dset)
    
display_metric(test_pred.metrics)

[2024-06-06 14:36:03,843] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


  0%|          | 0/196 [00:00<?, ?it/s]

node-0:248605:248605 [0] NCCL INFO Bootstrap : Using eth0:10.13.51.163<0>
node-0:248605:248605 [0] NCCL INFO NET/Plugin : Plugin load (librccl-net.so) returned 2 : librccl-net.so: cannot open shared object file: No such file or directory
node-0:248605:248605 [0] NCCL INFO NET/Plugin : No plugin found, using internal implementation
node-0:248605:248605 [0] NCCL INFO Kernel version: 5.15.0-1042-azure
RCCL version 2.17.1+hip5.7 HEAD:cbbb3d8+

node-0:248605:254246 [0] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

node-0:248605:254246 [0] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/transport/net_ib.cc:199 NCCL WARN NET/IB : Unable to open device mlx5_0

node-0:248605:254246 [0] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

node-0:248605:254

node-0:248605:254246 [0] NCCL INFO Ring 11 : 1 -> 0 -> 1 comm 0x18f28670 nRanks 02 busId d00000
node-0:248605:254246 [0] NCCL INFO Ring 12 : 1 -> 0 -> 1 comm 0x18f28670 nRanks 02 busId d00000
node-0:248605:254246 [0] NCCL INFO Ring 13 : 1 -> 0 -> 1 comm 0x18f28670 nRanks 02 busId d00000
node-0:248605:254246 [0] NCCL INFO Ring 14 : 1 -> 0 -> 1 comm 0x18f28670 nRanks 02 busId d00000
node-0:248605:254246 [0] NCCL INFO Ring 15 : 1 -> 0 -> 1 comm 0x18f28670 nRanks 02 busId d00000
node-0:248605:254246 [0] NCCL INFO Ring 16 : 1 -> 0 -> 1 comm 0x18f28670 nRanks 02 busId d00000
node-0:248605:254246 [0] NCCL INFO Ring 17 : 1 -> 0 -> 1 comm 0x18f28670 nRanks 02 busId d00000
node-0:248605:254246 [0] NCCL INFO Ring 18 : 1 -> 0 -> 1 comm 0x18f28670 nRanks 02 busId d00000
node-0:248605:254246 [0] NCCL INFO Ring 19 : 1 -> 0 -> 1 comm 0x18f28670 nRanks 02 busId d00000
node-0:248605:254246 [0] NCCL INFO Ring 20 : 1 -> 0 -> 1 comm 0x18f28670 nRanks 02 busId d00000
node-0:248605:254246 [0] NCCL INFO Ring 

/home/aiscuser/scratch/Projects/xcai/xcai/losses.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,31.9877,21.5409,16.2897,10.2778,31.9877,31.9262,32.9381,34.7662,26.4456,28.5595,30.6633,34.87,26.4456,28.8131,30.4457,32.5524,39.4305,52.5021,55.1797,0.0607,101.5535,1747.995,1.093


In [14]:
#| export
pred_dir = f'{mname}/predictions/'
os.makedirs(pred_dir, exist_ok=True)
with open(f'{pred_dir}/test_predictions.pkl', 'wb') as file: pickle.dump(test_pred, file)